In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import attila_utils

## Summary statistics

In [2]:
fpath = '../../resources/cell-based-assays/CO28151_outcome summary.xlsx'
sheet_name = 'CO28151'
CO28151 = pd.read_excel(fpath, sheet_name=sheet_name)

fpath = '../../resources/cell-based-assays/CO28151_outcome-summary_27062023-clean.xlsx'
sheet_name = 'CO28151'
CO28151_27062023 = pd.read_excel(fpath, sheet_name=sheet_name)

sheet_name = 'CO25360'
CO25360_27062023 = pd.read_excel(fpath, sheet_name=sheet_name)

### Assays

Assays in CO28151 before 2023 June 27

In [3]:
print(len(CO28151.assay.unique()), 'assays')
list(CO28151.assay.unique())

11 assays


['Abeta celarance supernatant',
 'Abeta celarance intracellular',
 'growth factor removal MTT',
 'growth factor removal LDH',
 'Abeta secretion 1-42',
 'Abeta secretion 1-40',
 'LPS inflammation TNFa',
 'LPS inflammation IL1b',
 'LPS inflammation IL6',
 'LPS inflammation KC/GRO',
 'LPS inflammation MTT']

Assays in CO28151 after 2023 June 27

In [4]:
print(len(CO28151_27062023.assay.unique()), 'assays')
list(CO28151_27062023.assay.unique())

14 assays


['Abeta celarance intracellular',
 'Abeta celarance supernatant',
 'Abeta secretion 1-40',
 'Abeta secretion 1-42',
 'growth factor removal LDH',
 'growth factor removal MTT',
 'LPS inflammation IL1b',
 'LPS inflammation IL6',
 'LPS inflammation KC/GRO',
 'LPS inflammation MTT',
 'LPS inflammation TNFa',
 'pTau 231',
 'pTau 231 vs total Tau',
 'Total Tau']

Assays in CO28151 added on 2023 June 27

In [5]:
set(CO28151_27062023.assay.unique()).difference(set(CO28151.assay.unique()))

{'Total Tau', 'pTau 231', 'pTau 231 vs total Tau'}

The same set of assays are contained in CO25360 as in CO28151

In [6]:
set(CO28151_27062023.assay.unique()) == set(CO25360_27062023.assay.unique())

True

### Treatments

Treatments in CO28151 before 2023 June 27

In [7]:
print(len(CO28151.TI.unique()), 'treatments')
treatments_CO28151 = pd.DataFrame(*[CO28151.groupby(['TI', 'name']).first().index.get_level_values(i) for i in [1, 0]])
treatments_CO28151

13 treatments


,name
TI,
TI1,Amiloride
TI11,DIM-C-pPhOCH3
TI12,C-DIM12
TI13,Liraglutide
TI14,Semaglutide
TI2,Phenamil methanesulfonate salt
TI3,Benzamil hydrochloride hydrate
TI4,"5-(N,N-Dimethyl)amiloride hydrochloride"
TI5,5-(N-Ethyl-N-isopropyl)amiloride


Treatments in CO28151 after on 2023 June 27

In [8]:
print(len(CO28151_27062023.TI.unique()), 'treatments')
treatments_CO28151_27062023 = pd.DataFrame(*[CO28151_27062023.groupby(['TI', 'name']).first().index.get_level_values(i) for i in [1, 0]])
treatments_CO28151_27062023

17 treatments


,name
TI,
TI1,Amiloride
TI10,Cilostazol
TI11,DIM-C-pPhOCH3
TI12,C-DIM12
TI13,Liraglutide
TI14,Semaglutide
TI15,Hydroxychloroquine racemic
TI16/17,"(S)Hydroxychloroquine racemic, free base"
TI18/19,"(R)Hydroxychloroquine racemic, free base"


Treatments in CO28151 added on 2023 June 27

In [9]:
treatments_CO28151_27062023.loc[list(set(treatments_CO28151_27062023.index).difference(set(treatments_CO28151.index)))]

,name
TI,
TI18/19,"(R)Hydroxychloroquine racemic, free base"
TI10,Cilostazol
TI16/17,"(S)Hydroxychloroquine racemic, free base"
TI15,Hydroxychloroquine racemic


Treatments in CO25360

In [10]:
print(len(CO25360_27062023.TI.unique()), 'treatments')
treatments_CO25360_27062023 = pd.DataFrame(*[CO25360_27062023.groupby(['TI', 'name']).first().index.get_level_values(i) for i in [1, 0]])
treatments_CO25360_27062023

27 treatments


,name
TI,
TI1,VALPROIC ACID
TI10,DASATINIB
TI11,FLUTICASONE
TI12,METHOTREXATE
TI13,HYDROXYCHLOROQUINE
TI14,ROFLUMILAST
TI15,NAPABUCASIN
TI16,CRIZOTINIB
TI17,C188-9


In [11]:
# concatenate the two studies
data = pd.concat([CO28151_27062023, CO25360_27062023], axis=0)
data = data.groupby(['name', 'assay']).first()['Mean Diff,'].unstack()
data

assay,Abeta celarance intracellular,Abeta celarance supernatant,Abeta secretion 1-40,Abeta secretion 1-42,LPS inflammation IL1b,LPS inflammation IL6,LPS inflammation KC/GRO,LPS inflammation MTT,LPS inflammation TNFa,Total Tau,growth factor removal LDH,growth factor removal MTT,pTau 231,pTau 231 vs total Tau
name,,,,,,,,,,,,,,
"(R)Hydroxychloroquine racemic, free base",-1.52100,13263.00,NaN,NaN,0.26110,278.1,0.09900,-4.333,99.60,NaN,NaN,NaN,NaN,NaN
"(S)Hydroxychloroquine racemic, free base",0.55500,715.80,NaN,NaN,0.23420,-2188.0,-1.05700,-3.500,43.60,NaN,NaN,NaN,NaN,NaN
"3,3'-Diindolylmethane",-7719.00000,4470.00,2216.00,226.20,-0.63800,233.0,3.88000,-0.517,204.00,-23828.0,-1.430,17.600,4.5000,0.07500
"5-(N,N-Dimethyl)amiloride hydrochloride",-0.59670,11579.00,1254891.00,70028.00,0.39330,14321.0,6.19700,96.500,1318.00,NaN,-36.330,38.670,NaN,NaN
"5-(N,N-Hexamethylene)amiloride",-1.81800,6853.00,-21838.00,-38713.00,1.32200,19211.0,9.16800,61.670,1385.00,NaN,-41.670,61.170,NaN,NaN
5-(N-Ethyl-N-isopropyl)amiloride,-3.17700,12536.00,957888.00,60425.00,1.28100,14353.0,8.40100,97.500,1384.00,NaN,-43.830,59.500,NaN,NaN
5-(N-Methyl-N-isobutyl)amiloride,-0.58500,8891.00,965683.00,60805.00,1.13200,19356.0,9.99200,77.170,1647.00,NaN,-45.670,60.000,NaN,NaN
A-317567,-1.44000,68649.00,NaN,NaN,0.45010,8922.0,4.04300,97.330,1394.00,NaN,-50.170,54.330,NaN,NaN
ACITRETIN,0.14200,-32.10,-1662.00,-218.00,-0.17800,83.3,-3.54000,46.700,-1371.00,-80305.0,-4.930,19.900,1.6700,2.21000


In [12]:
print(data.shape[0], 'treatments')
print(data.shape[1], 'assays')
print(data.shape[0], '*', data.shape[1], '=', data.shape[0] * data.shape[1], 'treatment-assay combinations')
print(data.isna().sum().sum(), '(', np.round(100 * data.isna().sum().sum() / 616), '% )', 'unmeasured combinations')

44 treatments
14 assays
44 * 14 = 616 treatment-assay combinations
72 ( 12.0 % ) unmeasured combinations


## Older 

## Raw data

In [13]:
def wide2long_form(batch, plate, assay):
    df = pd.read_clipboard()
    activity = df.stack().to_frame('Activity')
    batch = pd.DataFrame({'Batch': batch}, index=activity.index)
    plate = pd.DataFrame({'Plate': plate}, index=activity.index)
    assay = pd.DataFrame({'Assay': assay}, index=activity.index)
    middle = df.stack().index.to_frame().rename({0: 'Replica', 1: 'TI_conc'}, axis=1)
    TI = middle.TI_conc.apply(lambda s: re.sub('^(TI[0-9]{1,2}).*$', '\\1', s)).to_frame('TI')
    conc = middle.TI_conc.apply(lambda s: re.match('.*c[0-9]{1,2}$', s))
    conc = conc.apply(lambda x: re.sub('^(TI[0-1]{1,2}\\s+)(c[0-9]{1,2})$', '\\2', x.string) if x is not None else '').to_frame('conc')
    l = [batch, plate, TI, conc, assay, activity]
    long = pd.concat(l, axis=1)
    long.to_clipboard(index=False, header=False)
    return(long)

In [14]:
fpath = '../../resources/cell-based-assays/CO28151_data-summary-clean.xlsx'
data = dict()
key = 'Abeta Clearance'
data[key] = pd.read_excel(fpath, sheet_name=key)
data[key]

,Cohort,Group,IRN,TI,conc,Aβ42 SN,Aβ42 Lys,BCA,Aβ42 Lys pg per ug prot
0,plate1,A1,TI2 c1_1,TI2,C1,35,693.0,271.0,2.55
1,plate1,A1,TI2 c1_2,TI2,C1,102,948.0,454.0,2.09
2,plate1,A1,TI2 c1_3,TI2,C1,51,1885.0,474.0,3.98
3,plate1,A1,TI2 c1_4,TI2,C1,37,674.0,298.0,2.26
4,plate1,A1,TI2 c1_5,TI2,C1,55,430.0,336.0,1.28
...,...,...,...,...,...,...,...,...,...
300,Plate 1,VC1,VC_1_6,VC_1,NaN,60759,3023.0,371.0,8.15
301,Plate 2,VC2,VC_2_1,VC_2,NaN,64832,1492.0,409.0,3.65
302,Plate 2,VC2,VC_2_2,VC_2,NaN,53839,456.0,451.0,1.01
303,Plate 2,VC2,VC_2_3,VC_2,NaN,58911,195.0,410.0,0.48


In [15]:
key = 'Abeta secretion'
data[key] = pd.read_excel(fpath, sheet_name=key)
data[key]

,plate,Group,IRN,TI,conc,Aβ38,Aβ40,Aβ42,ratio 42:40x100
0,1,A1,TI2 c1_1,TI2,1.0,4010.000,834290.00,67940.0000,8.100000
1,1,A1,TI2 c1_2,TI2,1.0,6280.000,784080.00,70540.0000,9.000000
2,1,A1,TI2 c1_3,TI2,1.0,27790.000,932160.00,81130.0000,8.700000
3,1,A1,TI2 c1_4,TI2,1.0,32560.000,878050.00,69370.0000,7.900000
4,1,A1,TI2 c1_5,TI2,1.0,20670.000,624520.00,48540.0000,7.800000
...,...,...,...,...,...,...,...,...,...
277,5,NaN,TI15 c3_2,TI15,3.0,2538.531,14177.22,979.6982,6.910369
278,5,NaN,TI15 c3_3,TI15,3.0,2677.243,14218.80,1072.2790,7.541276
279,5,NaN,TI15 c3_4,TI15,3.0,2782.730,13733.23,1086.8330,7.913892
280,5,NaN,TI15 c3_5,TI15,3.0,2530.309,13473.46,1070.6570,7.946415


In [16]:
key = 'LPS inflammation'
data[key] = pd.read_excel(fpath, sheet_name=key)
data[key]

,Cohort,Group,IRN,TI,conc,IL-10,IL-1β,IL-6,KC/GRO,TNF-α
0,plate1,A1,TI2 c1_1,TI2,C1,1.46,1.25,1959,2.3,166
1,plate1,A1,TI2 c1_2,TI2,C1,3.23,2.08,3867,4.88,338
2,plate1,A1,TI2 c1_3,TI2,C1,2.81,2.29,4313,6.14,322
3,plate1,A1,TI2 c1_4,TI2,C1,2.57,1.75,4092,6.81,285
4,plate1,A1,TI2 c1_5,TI2,C1,3.15,2.26,2259,3.78,230
...,...,...,...,...,...,...,...,...,...,...
434,Plate 4,VC4,VC 4_2,VC 4,4,2.2,0.015,n.m.,n.m.,15
435,Plate 4,VC4,VC 4_3,VC 4,4,2.34,n.m.,n.m.,n.m.,18
436,Plate 4,VC4,VC 4_4,VC 4,4,1.94,n.m.,n.m.,n.m.,13
437,Plate 4,VC4,VC 4_5,VC 4,4,2.05,n.m.,n.m.,n.m.,19


In [17]:
key = 'Tau phosphorylation'
data[key] = pd.read_excel(fpath, sheet_name=key)
data[key]

,Batch,Plate,IRN,TI,Conc,Tau [pg/ml],pTau (T231) [AU],BCA total protein,Tau [pg/µg],pTau (T231) [AU2],Ratio pTau /Tau,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,batch 1,Plate 5,TI1 c1_1,TI1,1,508,62.4,240.3,2.1,29.5,14,NaN,NaN,NaN,NaN
1,batch 1,Plate 5,TI1 c1_2,TI1,1,322.8,47.6,185.6,1.7,27.4,15.7,NaN,NaN,NaN,NaN
2,batch 1,Plate 5,TI1 c1_3,TI1,1,338,45.9,158.3,2.1,21.5,10.1,NaN,NaN,NaN,NaN
3,batch 1,Plate 5,TI1 c1_4,TI1,1,339.6,49,161.8,2.1,23.4,11.1,NaN,NaN,NaN,NaN
4,batch 1,Plate 5,TI1 c1_5,TI1,1,334.2,47,320.3,1,45,43.1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,batch 1,Plate 5,VC_5_2,VC,5,735.3,101.9,218.3,3.4,30.2,9,NaN,NaN,NaN,NaN
362,batch 1,Plate 5,VC_5_3,VC,5,912.1,125,244.3,3.7,33.5,9,NaN,NaN,NaN,NaN
363,batch 1,Plate 5,VC_5_4,VC,5,747,110.2,222.6,3.4,32.8,9.8,NaN,NaN,NaN,NaN
364,batch 1,Plate 5,VC_5_5,VC,5,2078.7,222.9,196.1,10.6,21,2,NaN,NaN,NaN,NaN


In [18]:
groups = {k: v.TI.unique() for k, v in data.items()}
groups

{'Abeta Clearance': array(['TI2 ', 'TI3 ', 'TI4 ', 'TI5 ', 'TI6 ', 'TI7 ', 'TI1 ', 'TI11',
        'TI12', 'TI13', 'TI14', 'Abet', 'VC_1', 'VC_2', 'VC_3', 'TI8',
        'TI9', 'TI10', 'TI15'], dtype=object),
 'Abeta secretion': array(['TI2', 'TI3', 'TI4', 'VC', 'TI5', 'TI6', 'TI7', 'TI11', 'TI12',
        'TI13', 'TI14', 'TI1', 'RI', 'TI10', 'TI15'], dtype=object),
 'LPS inflammation': array(['TI2 ', 'TI3 ', 'TI4 ', 'TI5 ', 'TI6 ', 'TI7 ', 'TI1 ', 'TI11',
        'TI12', 'TI13', 'LPS ', 'TI14', 'LPS+', 'VC_1', 'VC_2', 'VC_3',
        'VC_4', 'VC_5', 'Treatment', 'TI 8', 'TI8 ', 'TI 9', 'TI9 ',
        'TI10', 'TI15', 'LPS 1', 'LPS 2', 'LPS 3', 'LPS 4', 'RI 1', 'RI 2',
        'RI 3', 'RI 4', 'VC 1', 'VC 2', 'VC 3', 'VC 4'], dtype=object),
 'Tau phosphorylation': array(['TI1 ', 'TI2', 'TI3', 'TI4', 'TI5', 'TI6', 'TI7', 'TI8', 'TI9',
        'TI10', 'TI11', 'TI12', 'TI13', 'TI14', 'TI15', 'VC', 'RI'],
       dtype=object)}

In [19]:
treatments = dict()
key = 'Abeta Clearance'
treatments[key] = [x for x in groups[key] if re.match('^TI.*', x)]
print(treatments[key], len(treatments[key]))

['TI2 ', 'TI3 ', 'TI4 ', 'TI5 ', 'TI6 ', 'TI7 ', 'TI1 ', 'TI11', 'TI12', 'TI13', 'TI14', 'TI8', 'TI9', 'TI10', 'TI15'] 15


In [20]:
key = 'Abeta secretion'
treatments[key] = [x for x in groups[key] if isinstance(x, int)]
print(treatments[key], len(treatments[key]))

[] 0


In [21]:
treatments = dict()
key = 'LPS inflammation'
treatments[key] = [x for x in groups[key] if re.match('^TI[0-9]+', x)]
print(treatments[key], len(treatments[key]))

['TI2 ', 'TI3 ', 'TI4 ', 'TI5 ', 'TI6 ', 'TI7 ', 'TI1 ', 'TI11', 'TI12', 'TI13', 'TI14', 'TI8 ', 'TI9 ', 'TI10', 'TI15'] 15


In [22]:
treatments = dict()
key = 'Tau phosphorylation'
treatments[key] = [x for x in groups[key] if x not in ['VC', 'RI']]
print(treatments[key], len(treatments[key]))

['TI1 ', 'TI2', 'TI3', 'TI4', 'TI5', 'TI6', 'TI7', 'TI8', 'TI9', 'TI10', 'TI11', 'TI12', 'TI13', 'TI14', 'TI15'] 15


In [23]:
%connect_info

{
  "shell_port": 53569,
  "iopub_port": 53570,
  "stdin_port": 53571,
  "control_port": 53573,
  "hb_port": 53572,
  "ip": "127.0.0.1",
  "key": "78c6d363-8ebdeb95a14c3ea0fadca073",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "python3"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-d8d40d23-56e5-451f-9dc8-50d2a720446d.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
